In [1]:
import pandas as pd
import scipy as sp
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats.anova import anova_lm
import scipy.stats as stats

In [80]:
metabolites = pd.read_csv("/Users/huananshi/Desktop/plasma_nutrients.csv",index_col=0)
metabolites = metabolites.transpose()
metabolites_log10 = metabolites.apply(np.log10)
metadata = pd.read_csv("/Users/huananshi/Box Sync/EODF_analysis/intermediate_files/metadata/metadata.csv",index_col=0)
metabolites_sample = metadata.index.intersection(metabolites.index)
#gene_name = gene.loc["description"]
metabolites_metadata_log10 = pd.concat([metadata.loc[metabolites_sample],metabolites_log10.loc[metabolites_sample]],sort = True, axis =1 )
metabolites_metadata = pd.concat([metadata.loc[metabolites_sample],metabolites.loc[metabolites_sample]],sort = True, axis =1 )

               
                                 

In [81]:
def metabolite_stat(result):
    #there are a ton of stuff can be simplified or optimized for more general use: I can load the file and filter groups based on given names
    #i can set an argument to determine the numbers of groups, factors and determine which stats should be run
    res_sca=result  
    metabolites = res_sca.columns.tolist()[4:]
    res_stat = pd.DataFrame()
    res_mc = pd.DataFrame()

    for metabolite in metabolites: 
        test = res_sca.loc[:,["Genotype", "Group","Genotype_Group",metabolite]]
        test.columns = ["Genotype", "Group","Genotype_Group","value"]
        test.loc[:,"value"]=test.loc[:,"value"].astype(float)
        model = ols('value ~ C(Genotype)*C(Group)', test).fit()
        aov_table = anova_lm(model, typ=2)
        res_stat=res_stat.append(aov_table["PR(>F)"].transpose()[0:3])
        mc = statsmodels.stats.multicomp.MultiComparison(test['value'],test['Genotype_Group'])
        mc_BH_table = mc.allpairtest(stats.ttest_ind,method='fdr_bh')[0]
        mc_table=pd.DataFrame(mc_BH_table.data[1:],columns=mc_BH_table.data[0])
        res_mc = res_mc.append(mc_table["pval_corr"].transpose())
    res_mc.index = metabolites
    res_stat.index = metabolites
    res_mc.columns = mc_table["group1"]+"_"+mc_table["group2"]
    res_stats = pd.concat([res_stat,res_mc],axis =1)
    return res_stats #,res_sca_sig

In [82]:
metabolites_abun_stat = metabolite_stat(metabolites_metadata_log10)

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:325: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha


In [83]:
metabolites_names = metabolites.columns
metabolites_sum = metabolites_metadata.groupby("Genotype_Group").mean().transpose()
metabolites_abun_stat['log2fc_SC_WC'] = metabolites_sum["SHRSP_Con"].div(metabolites_sum["WKY_Con"]).apply(np.log2)
metabolites_abun_stat['log2fc_SC_SE'] = metabolites_sum["SHRSP_Con"].div(metabolites_sum["SHRSP_EODF"]).apply(np.log2)
metabolites_abun_stat['fc_SC_WC'] = metabolites_sum["SHRSP_Con"].div(metabolites_sum["WKY_Con"])
metabolites_abun_stat['fc_SC_SE'] = metabolites_sum["SHRSP_Con"].div(metabolites_sum["SHRSP_EODF"])

In [85]:
metabolites_abun_stat.to_csv("/Users/huananshi/Desktop/plasma_nutrients_stats.csv")